# Build Neural Network definition from code

In this example we should how to build a `NetworkDefinition` directly from Python.

In [1]:
import pyomo.environ as pyo
import numpy as np

from omlt.block import OmltBlock
from omlt.neuralnet import NetworkDefinition, InputLayer, DenseLayer, ReLUBigMFormulation, IndexTransformer

In [2]:
net = NetworkDefinition(input_bounds=[(-1.0, 1.0)] * 4)

# 2x2 input layer
input_layer = InputLayer([2, 2])
net.add_node(input_layer)

# dense layer, [2, 2] -> [2, 1]
dense_layer_0 = DenseLayer(
    input_size=input_layer.output_size,
    output_size=[2, 1],
    activation="linear",
    weights=np.array([[1.0], [-0.5]]),
    biases=np.array([[0.1], [0.25]])
)
net.add_node(dense_layer_0)
net.add_edge(input_layer, dense_layer_0)

# dense layer, [2] -> [1]
# notice the use of the IndexTransformer to "flatten" the output of dense_layer_0
dense_layer_1 = DenseLayer(
    input_size=[2],
    output_size=[1],
    activation="relu",
    weights=np.array([2.0, -1.0]),
    biases=np.array([0.0]),
    input_index_transformer=IndexTransformer([2, 1], [2])
)
net.add_node(dense_layer_1)
net.add_edge(dense_layer_0, dense_layer_1)

Layers provide a function `eval` to evaluate the layer with the given input. The input must be a numpy array of `layer.input_size` size.

In [3]:
x = np.diag([1.0, 0.1]) + 0.3

for layer in net.nodes:
    x = layer.eval(x)
    print(layer)
    print(x)
    print()
    

InputLayer(input_size=[2, 2], output_size=[2, 2])
[[1.3 0.3]
 [0.3 0.4]]

DenseLayer(input_size=[2, 2], output_size=[2, 1])
[[1.25]
 [0.35]]

DenseLayer(input_size=[2], output_size=[1])
[2.15]



In [5]:
m = pyo.ConcreteModel()
m.neural_net = OmltBlock()
formulation = ReLUBigMFormulation(net)
m.neural_net.build_formulation(formulation)

In [7]:
m.neural_net.inputs.pprint()

inputs : Size=4, Index=neural_net.inputs_set
    Key    : Lower : Value : Upper : Fixed : Stale : Domain
    (0, 0) :  -1.0 :     0 :   1.0 : False : False :  Reals
    (0, 1) :  -1.0 :     0 :   1.0 : False : False :  Reals
    (1, 0) :  -1.0 :     0 :   1.0 : False : False :  Reals
    (1, 1) :  -1.0 :     0 :   1.0 : False : False :  Reals


In [8]:
m.neural_net.outputs.pprint()

outputs : Size=1, Index=neural_net.outputs_set
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      0 :  None :     0 :  None : False : False :  Reals


In [9]:
m.neural_net.layer[0].pprint()

{Member of layer} : Size=3, Index=neural_net.layer_index, Active=True
    neural_net.layer[0] : Active=True
        3 Set Declarations
            constraints_index : Size=1, Index=None, Ordered=Insertion
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    0 :      {}
            z_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     2 :    Any :    4 : {(0, 0), (0, 1), (1, 0), (1, 1)}
            zhat_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     2 :    Any :    4 : {(0, 0), (0, 1), (1, 0), (1, 1)}

        2 Var Declarations
            z : Size=4, Index=neural_net.layer[0].z_index
                Key    : Lower : Value : Upper : Fixed : Stale : Domain
                (0, 0) :  -1.0 :     0 :   1.0 : False : False :  Reals
                (0, 1) :  -1.0 :     0 :   1.0 : False : False :  Real

In [10]:
m.neural_net.layer[1].pprint()

{Member of layer} : Size=3, Index=neural_net.layer_index, Active=True
    neural_net.layer[1] : Active=True
        4 Set Declarations
            _linear_activation_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     2 :    Any :    2 : {(0, 0), (1, 0)}
            constraints_index : Size=1, Index=None, Ordered=Insertion
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    2 : {1, 2}
            z_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     2 :    Any :    2 : {(0, 0), (1, 0)}
            zhat_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     2 :    Any :    2 : {(0, 0), (1, 0)}

        2 Var Declarations
            z : Size=2, Index=neural_net.layer[1].z_index
                Key    : Lower  : Value : Upper : Fixed

In [11]:
m.neural_net.layer[2].pprint()

{Member of layer} : Size=3, Index=neural_net.layer_index, Active=True
    neural_net.layer[2] : Active=True
        10 Set Declarations
            _big_m_lb_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    1 :    {0,}
            _big_m_ub_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    1 :    {0,}
            _z_lower_bound_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    1 :    {0,}
            _z_lower_bound_zhat_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     1 :    Any :    1 :    {0,}
            _z_upper_bound_index : Size=1, Index=None, Ordered=False
                Key  : Dimen : Domain : Size : Members
                None :     1 